In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd
from rich.progress import track
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

sys.path.append("../src")
from utils.feature import FeatureStore
from utils.common import load_pickle, dump_pickle, save_cache

In [2]:
from dataset import IndoorDataset, IndoorDataModule
from submission import IndoorTestDataset

In [3]:
d1 = torch.zeros((10))
d2 = torch.ones((10)) * 2

print(d1)
print(d2)

loss_fn = nn.MSELoss()
loss_fn(d1, d2)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])


tensor(4.)

In [4]:
torch.mean(torch.pow(torch.abs(d1 - d2), 2))

tensor(4.)

In [5]:
d1 = torch.tensor(
    [
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
    ], dtype=torch.float)
d2 = torch.tensor(
    [
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
    ], dtype=torch.float)

print(d1)
print(d2)

loss_fn = nn.MSELoss()
loss_fn(d1, d2)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.],
        [2., 5.]])


tensor(8.5000)

In [16]:
diff = torch.abs(d1 - d2)
diff

tensor([[1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.],
        [1., 4.]])

In [17]:
tmp = torch.sum(diff, dim=1)
tmp

tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [10]:
torch.mean(tmp)

tensor(5.)

In [11]:
class MSELoss(nn.Module):
    def __init__(self):
        super(MSELoss, self).__init__()

    def forward(self, y_hat, y):
        pos_error = y_hat - y
        pos_error = torch.sum(torch.abs(pos_error), dim=1)
        return torch.mean(pos_error)

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, y_hat, y):
        pos_error = y_hat - y
        pos_error = torch.sum(torch.sqrt(torch.pow(pos_error, 2)), dim=1)
        return torch.mean(pos_error)


class MeanPositionLoss(nn.Module):
    def __init__(self):
        super(MeanPositionLoss, self).__init__()

    def forward(self, y_hat, y, floor_hat, floor):
        pos_error = y_hat - y
        pos_error = torch.sum(torch.sqrt(torch.pow(pos_error, 2)), dim=1)
        
        p = 15
        floor_error = p * torch.abs(floor_hat - floor)
        
        error = pos_error + floor_error
        return torch.mean(error)

In [12]:
# class MeanPositionLoss(nn.Module):
#     def __init__(self):
#         super(MeanPositionLoss, self).__init__()

#     def forward(self, y_hat, y):
#         pos = y
#         pos_hat = y_hat

#         diff_x = pos_hat[:, 0] - pos[:, 0]
#         diff_y = pos_hat[:, 1] - pos[:, 1]

#         error = torch.sqrt(torch.pow(diff_x, 2) + torch.pow(diff_y, 2))
#         return torch.mean(error)

In [13]:
loss_fn = MeanPositionLoss()
loss_fn(d1, d2, torch.zeros(10), torch.zeros(10))

tensor(5.)

In [14]:
loss_fn = RMSELoss()
loss_fn(d1, d2)

tensor(5.)

In [15]:
loss_fn = MSELoss()
loss_fn(d1, d2)

tensor(5.)